In [1]:
import pandas as pd
import numpy as np
import random
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# -------------------------------------------------------------------------
# 1. 데이터 불러오기 (기본 전처리된 파일 사용)
# -------------------------------------------------------------------------
data_path = '../data/ml-1m'

# (이전에 저장해둔 전처리된 파일이 있다고 가정)
users = pd.read_csv(f'{data_path}/users_prepro.csv')
movies = pd.read_csv(f'{data_path}/movies_prepro.csv')
ratings = pd.read_csv(f'{data_path}/ratings_prepro.csv')

# timestamp가 문자열이라면 정수 변환 (혹시 모를 에러 방지)
# ratings_prepro.csv에 이미 rating_year 등이 있다면 활용, 없다면 생성
if 'rating_year' not in ratings.columns:
    ratings['timestamp'] = pd.to_datetime(ratings['timestamp'])
    ratings['rating_year'] = ratings['timestamp'].dt.year
    ratings['rating_month'] = ratings['timestamp'].dt.month

print("데이터 로드 완료.")


데이터 로드 완료.


In [2]:

# -------------------------------------------------------------------------
# 2. Hard Negative Sampling 준비 (영화 인기도 계산)
# -------------------------------------------------------------------------
# 평점 개수가 많은 순서대로 영화 ID를 정렬 (전체 데이터 기준)
popular_movie_ids = ratings.groupby('movie_id').size().sort_values(ascending=False).index.tolist()

# 상위 2000개를 "인기 영화"로 정의
top_popular_movies = set(popular_movie_ids[:2000])


In [3]:

# -------------------------------------------------------------------------
# 3. Positive 데이터 생성 (평점 4점 이상 = 1)
# -------------------------------------------------------------------------
# 4점 이상을 선호(Positive)로 간주
positive_ratings = ratings[ratings['rating'] >= 4].copy()
positive_ratings['label'] = 1
# 필요한 컬럼만 유지
positive_ratings = positive_ratings[['user_id', 'movie_id', 'rating_year', 'rating_month', 'label']]

print(f"Positive 데이터 수: {len(positive_ratings)}")


Positive 데이터 수: 575281


In [4]:

# -------------------------------------------------------------------------
# 4. Negative 데이터 생성 (Hard Negative + Random Negative)
# -------------------------------------------------------------------------
# 사용자가 봤던(선호했던) 영화 리스트 추출
user_seen_movies = ratings.groupby('user_id')['movie_id'].apply(list).reset_index()

unique_movies = movies['movie_id'].unique()
unique_users = users['user_id'].unique()
negative_users = []
negative_movies = []
negative_labels = []

print("Hard Negative Sampling 시작...")

for user in unique_users:
    # 4-1. 해당 사용자가 선호하는 영화 리스트 (Set)
    seen_series = user_seen_movies[user_seen_movies['user_id'] == user]['movie_id']
    if len(seen_series) < 1:
        continue
        
    user_seen_movie_list = set(seen_series.values[0])
    
    # 4-2. 사용자가 안 본 영화 전체 리스트
    all_movies_set = set(unique_movies)
    user_non_seen_movie_set = all_movies_set - user_seen_movie_list
    
    # 4-3. 샘플링 개수 설정 (Positive 개수의 5배, 데이터가 부족하면 가능한 만큼만)
    # user_seen_movie_list는 전체 시청 이력이 아니라 '선호' 이력일 수 있으므로
    # positive_ratings에서 해당 유저의 개수를 세는 것이 더 정확할 수 있으나,
    # 여기서는 기존 로직 유지 (시청 이력 기준)
    total_negative_num = len(user_seen_movie_list) * 5
    if len(user_non_seen_movie_set) < total_negative_num:
        total_negative_num = len(user_non_seen_movie_set)
    
    # 4-4. Hard vs Random 비율 설정 (5:5)
    hard_negative_num = int(total_negative_num * 0.5)
    random_negative_num = total_negative_num - hard_negative_num
    
    # A. Hard Negative 추출 (안 본 영화 & 인기 영화)
    hard_candidates = list(top_popular_movies & user_non_seen_movie_set)
    
    if len(hard_candidates) < hard_negative_num:
        selected_hard = hard_candidates
        random_negative_num += (hard_negative_num - len(hard_candidates))
    else:
        selected_hard = random.sample(hard_candidates, hard_negative_num)
        
    # B. Random Negative 추출
    user_non_seen_movie_list = list(user_non_seen_movie_set)
    selected_random = random.sample(user_non_seen_movie_list, random_negative_num)
    
    # C. 합치기
    user_negative_movie_list = selected_hard + selected_random
    
    # 결과 저장
    negative_users += [user for _ in range(len(user_negative_movie_list))]
    negative_movies += user_negative_movie_list
    negative_labels += [0 for _ in range(len(user_negative_movie_list))]

print(f"Negative Sampling 완료! (샘플 수: {len(negative_users)})")

# Negative DataFrame 생성
negative_ratings_df = pd.DataFrame({
    'user_id': negative_users, 
    'movie_id': negative_movies, 
    'label': negative_labels
})

# Negative 데이터에는 rating_year 정보가 없으므로, 
# 해당 유저의 평균 rating_year나 최신 year, 혹은 0으로 채워야 함.
# 여기서는 가장 많이 등장한 연도(mode)나 임의의 값(2000)으로 채웁니다.
negative_ratings_df['rating_year'] = 2000 
negative_ratings_df['rating_month'] = 1


Hard Negative Sampling 시작...
Negative Sampling 완료! (샘플 수: 4712069)


In [5]:

# -------------------------------------------------------------------------
# 5. 데이터 병합 (Positive + Negative)
# -------------------------------------------------------------------------
final_ratings = pd.concat([positive_ratings, negative_ratings_df], axis=0)

# 유저/영화 정보 병합
merge_data = pd.merge(final_ratings, movies, on='movie_id')
merge_data = pd.merge(merge_data, users, on='user_id')


In [6]:

# -------------------------------------------------------------------------
# 6. 피처 엔지니어링 (파생 변수 생성)
# -------------------------------------------------------------------------
print("피처 엔지니어링 진행 중...")

# 6-1. 통계형 피처 (영화 인기도, 유저 활동성) - 수치형
# 전체 ratings(원본) 기준으로 계산해야 정확함
movie_counts = ratings.groupby('movie_id')['rating'].count()
user_counts = ratings.groupby('user_id')['rating'].count()

merge_data['movie_pop'] = merge_data['movie_id'].map(movie_counts).fillna(0)
merge_data['user_act'] = merge_data['user_id'].map(user_counts).fillna(0)

# 수치형 피처 구간화 (Binning) -> AutoInt는 범주형 입력을 선호하므로
merge_data['movie_pop'] = pd.qcut(merge_data['movie_pop'], 10, labels=False, duplicates='drop')
merge_data['user_act'] = pd.qcut(merge_data['user_act'], 10, labels=False, duplicates='drop')

# 6-2. 시간형 피처 (개봉 후 경과 기간)
# movie_year 컬럼이 문자열이면 숫자 변환 필요
if merge_data['movie_year'].dtype == object:
    merge_data['movie_year'] = pd.to_numeric(merge_data['movie_year'], errors='coerce').fillna(2000).astype(int)

merge_data['release_lag'] = merge_data['rating_year'] - merge_data['movie_year']
merge_data['release_lag'] = merge_data['release_lag'].apply(lambda x: max(0, x)) # 음수 제거
merge_data['release_lag'] = merge_data['release_lag'].apply(lambda x: x // 5) # 5년 단위 구간화

# 6-3. 교차 피처 (Cross Feature) - age_gender
merge_data['age_gender'] = merge_data['age'].astype(str) + "_" + merge_data['gender'].astype(str)
merge_data['age_genre'] = merge_data['age'].astype(str) + "_" + merge_data['genre1'].astype(str)


피처 엔지니어링 진행 중...


In [7]:

# -------------------------------------------------------------------------
# 7. 결측치 처리 및 저장
# -------------------------------------------------------------------------
merge_data.fillna('no', inplace=True)

# 저장할 컬럼 순서 정리
target_columns = [
    'user_id', 'movie_id', 
    'movie_decade', 'movie_year', 
    'rating_year', 'rating_month', 
    'genre1', 'genre2', 'genre3', 
    'gender', 'age', 'occupation', 'zip',
    'movie_pop', 'user_act', 'release_lag', # 추가된 통계/시간 피처
    'age_gender', 'age_genre',              # 추가된 교차 피처
    'label'
]

# 컬럼 필터링 (없는 컬럼은 제외)
final_columns = [col for col in target_columns if col in merge_data.columns]
final_df = merge_data[final_columns]

print(f"최종 데이터 크기: {final_df.shape}")
print(final_df.head())

# CSV 저장
save_path = f'{data_path}/movielens_rcmm_v4.csv'
final_df.to_csv(save_path, index=False)
print(f"저장 완료: {save_path}")

최종 데이터 크기: (5287350, 19)
   user_id  movie_id movie_decade  movie_year  rating_year  rating_month  \
0        1      1193        1970s        1975         2001             1   
1        1      3408        2000s        2000         2001             1   
2        1      2355        1990s        1998         2001             1   
3        1      1287        1950s        1959         2001             1   
4        1      2804        1980s        1983         2001             1   

      genre1      genre2  genre3 gender  age  occupation    zip  movie_pop  \
0      Drama          no      no      F    1          10  48067          9   
1      Drama          no      no      F    1          10  48067          9   
2  Animation  Children's  Comedy      F    1          10  48067          9   
3     Action   Adventure   Drama      F    1          10  48067          8   
4     Comedy       Drama      no      F    1          10  48067          9   

   user_act  release_lag age_gender    age_genre 